# 18장 몬테 카를로 시뮬레이션

<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/rickiepark/python4daml/blob/main/18장.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="코랩에서 실행하기"/></a>
</td></tr></table>

## 18.1 파스칼의 문제

In [3]:
import random 
import numpy as np 

In [4]:
def roll_die(): 
    return random.choice([1,2,3,4,5,6]) 

def check_pascal(num_trials): 
    """num_trials은 int > 0라고 가정합니다.
       이길 확률을 출력합니다""" 
    num_wins = 0 
    for i in range(num_trials): 
        for j in range(24): 
            d1 = roll_die() 
            d2 = roll_die() 
            if d1 == 6 and d2 == 6: 
                num_wins += 1 
                break 
    print('이길 확률 =', num_wins/num_trials) 

In [5]:
check_pascal(1000000)

이길 확률 = 0.491067


## 18.2 Pass 또는 Don’t pass?

그림 18-2 `Craps_game` 클래스

In [6]:
class Craps_game(object): 
    def __init__(self): 
        self.pass_wins, self.pass_losses = 0, 0 
        self.dp_wins, self.dp_losses, self.dp_pushes = 0, 0, 0 

    def play_hand(self): 
        throw = roll_die() + roll_die() 
        if throw == 7 or throw == 11: 
            self.pass_wins += 1 
            self.dp_losses += 1 
        elif throw == 2 or throw == 3 or throw == 12: 
            self.pass_losses += 1 
            if throw == 12: 
                self.dp_pushes += 1 
            else: 
                self.dp_wins += 1 
        else: 
            point = throw 
            while True: 
                throw = roll_die() + roll_die() 
                if throw == point: 
                    self.pass_wins += 1 
                    self.dp_losses += 1 
                    break 
                elif throw == 7: 
                    self.pass_losses += 1 
                    self.dp_wins += 1 
                    break 

    def pass_results(self): 
        return (self.pass_wins, self.pass_losses) 

    def dp_results(self): 
        return (self.dp_wins, self.dp_losses, self.dp_pushes) 

그림 18-3 크랩스 게임 시뮬레이션하기

In [7]:
def craps_sim(hands_per_game, num_games): 
    """hands_per_game과 num_games는 ints > 0이라고 가정합니다.
       hands_per_game번 핸드를 반복하는 num_games번 게임을 플레이하고 결과를 출력합니다"""
    games = [] 

    #num_games번 게임을 플레이합니다
    for t in range(num_games): 
        c = Craps_game() 
        for i in range(hands_per_game): 
            c.play_hand() 
        games.append(c) 

    #각 게임에 대한 통계를 생성합니다
    p_ROI_per_game, dp_ROI_per_game = [], [] 
    for g in games: 
        wins, losses = g.pass_results() 
        p_ROI_per_game.append((wins -losses)/float(hands_per_game)) 
        wins, losses, pushes = g.dp_results() 
        dp_ROI_per_game.append((wins -losses)/float(hands_per_game)) 

    #요약 통계를 만들어 출력합니다
    mean_ROI = str(round((100*sum(p_ROI_per_game)/num_games), 4)) + '%' 
    sigma = str(round(100*np.std(p_ROI_per_game), 4)) + '%' 
    print('패스:', '평균 ROI =', mean_ROI, '표준 편차 =', sigma) 
    mean_ROI = str(round((100*sum(dp_ROI_per_game)/num_games), 4)) +'%' 
    sigma = str(round(100*np.std(dp_ROI_per_game), 4)) + '%' 
    print('돈 패스:','평균 ROI =', mean_ROI, '표준 편차 =', sigma) 

In [8]:
craps_sim(20, 10)

패스: 평균 ROI = -8.0% 표준 편차 = 21.8174%
돈 패스: 평균 ROI = 4.5% 표준 편차 = 22.6329%
